# PyCitySchools

In [1]:
# dependencies
import pandas as pd
import os

In [2]:
# set file paths
schoolsPath = os.path.join("Resources", "schools_complete.csv")
studentsPath = os.path.join("Resources", "students_complete.csv")

# read in csv files
schools = pd.read_csv(schoolsPath)
students = pd.read_csv(studentsPath)

## Schools Data

In [3]:
#schools.head()

## Students Data

In [4]:
#students.head()

In [5]:
# combine both files into one csv dataset
schoolsNstudents = pd.merge(students, schools, on="school_name", how="left")

## Combined Data

In [6]:
#schoolsNstudents.head()

### Calculations for District Summary

In [7]:
# calculate the total number of schools
total_schools = schools["school_name"].count()
#total_schools

In [8]:
# calculate the total number of students
total_students = students["student_name"].count()
#total_students

In [9]:
# calculate the total budget
total_budget = schools["budget"].sum()
#total_budget

In [10]:
# calculate the average math score
average_math = students["math_score"].mean()
#average_math

In [11]:
# calculate the average reading score
average_reading = students["reading_score"].mean()
#average_reading

In [12]:
# calculate the percentage of students with a passing math score (70 or greater)
math_greater_70 = students.loc[students["math_score"] >= 70, :]
math_passing_percent = (len(math_greater_70) / total_students)*100
#math_passing_percent

In [13]:
# calculate the percentage of students with a passing reading score (70 or greater)
reading_greater_70 = students.loc[students["reading_score"] >= 70, :]
reading_passing_percent = (len(reading_greater_70) / total_students)*100
#reading_passing_percent

In [14]:
# calculate the percentage of students who passed math and reading (% Overall Passing)
passing_math_reading = students.loc[(students["math_score"] >= 70) & (students["reading_score"] >= 70), :]
overall_passing_percent = (len(passing_math_reading) / total_students)*100
#overall_passing_percent

In [15]:
# Create a dataframe to hold the above results
district_summary = pd.DataFrame({"Total Schools": [total_schools],
                        "Total Students": [total_students],
                        "Total Budget": [total_budget],
                        "Average Math Score": [average_math],
                        "Average Reading Score": [average_reading],
                        "% Passing Math": [math_passing_percent],
                        "% Passing Reading": [reading_passing_percent],
                        "% Overall Passing": [overall_passing_percent],
                        })

In [16]:
# formatting for better read of district summary
district_summary["Total Students"] = district_summary["Total Students"].map('{:,.0f}'.format)
district_summary["Total Budget"] = district_summary["Total Budget"].map('${:,.2f}'.format)

## District Summary

In [17]:
#district_summary

### Calculations for School Summary

In [18]:
# calculate budget per student by schools
schools["Per Student Budget"] = (schools["budget"] / schools["size"])
schools = schools.sort_values("school_name", ascending=True).reset_index(drop=True)
#schools["Per Student Budget"] = schools["Per Student Budget"].astype(float)
#schools

In [19]:
# calculate the average math scores by schools
math_score_per_school = students.groupby("school_name") ["math_score"].sum()
students_per_school = students.groupby("school_name")["math_score"].count()
average_math_score_df = math_score_per_school / students_per_school
#average_math_score_df

In [20]:
# calculate the average reading scores by schools
reading_score_per_school = students.groupby("school_name") ["reading_score"].sum()
average_reading_score_df = reading_score_per_school / students_per_school
#average_reading_score_df

In [21]:
# calculate the percentage of students with a passing math score (70 or greater) per school
math_greater_70_per_school = students.loc[students["math_score"] >= 70, :]
group_math_greater_70_per_school = math_greater_70_per_school.groupby("school_name") ["math_score"].count()
math_passing_pecent_per_school_df = (group_math_greater_70_per_school / students_per_school) * 100
#math_passing_pecent_per_school_df

In [22]:
# calculate the percentage of students with a passing reading score (70 or greater) per school
reading_greater_70_per_school = students.loc[students["reading_score"] >= 70, :]
group_reading_greater_70_per_school = reading_greater_70_per_school.groupby("school_name") ["reading_score"].count()
reading_passing_pecent_per_school_df = (group_reading_greater_70_per_school / students_per_school) * 100
#reading_passing_pecent_per_school_df

In [23]:
# calculate the percentage of students who passed math and reading (% Overall Passing) per school
passing_math_reading_per_school = students.loc[(students["math_score"] >= 70) & (students["reading_score"] >= 70), :]
group_passing_math_reading_per_school = passing_math_reading_per_school.groupby("school_name") ["school_name"].count()
overall_passing_percent_per_school_df = (group_passing_math_reading_per_school / students_per_school)*100
#overall_passing_percent_per_school_df

In [24]:
# Collect all require student marks df and place in a summary dataframe
student_marks_summary_df = pd.DataFrame({"Average Math Score": average_math_score_df,
                                         "Average Reading Score": average_reading_score_df,
                                         "% Passing Math": math_passing_pecent_per_school_df,
                                         "% Passing Reading": reading_passing_pecent_per_school_df,
                                         "% Overall Passing": overall_passing_percent_per_school_df
                                          }).reset_index()
#student_marks_summary_df

In [25]:
# merge the schools and the student marks summary for school summary output; along with viewable formats
school_summary = pd.merge(schools, student_marks_summary_df, on="school_name", how="left")

del school_summary["School ID"]

school_summary = school_summary.rename(columns={"school_name": "School Name", 
                                                "type": "School Type",
                                                "size": "Total Students",
                                                "budget": "Total School Budget"                                    
                                                })

school_summary["Total Students"] = school_summary["Total Students"].map('{:,.0f}'.format)
school_summary["Total School Budget"] = school_summary["Total School Budget"].map('${:,.2f}'.format)
school_summary["Per Student Budget"] = school_summary["Per Student Budget"].map('${:.2f}'.format)

## School Summary

In [26]:
#school_summary.head()

In [27]:
# Top 5 Performing Schools (By % Overall Passing)
school_summary_sort = school_summary.sort_values("% Overall Passing", ascending=False).reset_index(drop=True)

## Top 5 Performing Schools (By % Overall Passing)

In [28]:
#school_summary_sort.head()

In [29]:
#Bottom 5 Performing Schools (By % Overall Passing)
school_summary_sort = school_summary.sort_values("% Overall Passing", ascending=True).reset_index(drop=True)

## Bottom 5 Performing Schools (By % Overall Passing) 

In [30]:
#school_summary_sort.head()

### Script for math scores by grade by school

In [31]:
# Math Scores by Grade
# Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.
grade9_math_scores = (students.loc[students["grade"] == "9th"]).groupby("school_name")["math_score"].mean()
grade10_math_scores = (students.loc[students["grade"] == "10th"]).groupby("school_name")["math_score"].mean()
grade11_math_scores = (students.loc[students["grade"] == "11th"]).groupby("school_name")["math_score"].mean()
grade12_math_scores = (students.loc[students["grade"] == "12th"]).groupby("school_name")["math_score"].mean()
#grade12_math_scores

In [32]:
# Collect all require student math marks df by grade by school and place in a summary dataframe
math_scores_per_grade_summary_df = pd.DataFrame({"Grade 9 Math Avg": grade9_math_scores,
                                         "Grade 10 Math Avg": grade10_math_scores,
                                         "Grade 11 Math Avg": grade11_math_scores,
                                         "Grade 12 Math Avg": grade12_math_scores
                                          }).reset_index()

## Math Scores by Grade by Schools

In [33]:
#math_scores_per_grade_summary_df

### Script for reading scores by grade by school

In [34]:
# Reading Scores by Grade
# Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.
grade9_reading_scores = (students.loc[students["grade"] == "9th"]).groupby("school_name")["reading_score"].mean()
grade10_reading_scores = (students.loc[students["grade"] == "10th"]).groupby("school_name")["reading_score"].mean()
grade11_reading_scores = (students.loc[students["grade"] == "11th"]).groupby("school_name")["reading_score"].mean()
grade12_reading_scores = (students.loc[students["grade"] == "12th"]).groupby("school_name")["reading_score"].mean()
#grade12_reading_scores

In [35]:
# Collect all require student reading marks df by grade by school and place in a summary dataframe
reading_scores_per_grade_summary_df = pd.DataFrame({"Grade 9 Reading Avg": grade9_reading_scores,
                                         "Grade 10 Avg Reading Avg": grade10_reading_scores,
                                         "Grade 11 Avg Reading Avg": grade11_reading_scores,
                                         "Grade 12 Avg Reading Avg": grade12_reading_scores
                                          }).reset_index()

## Reading Scores by Grade by Schools

In [36]:
#reading_scores_per_grade_summary_df

### Binning to get scores by school spending

In [37]:
# build another merge school summary dataframe for binning with excepted unformatted dtypes
school_summary_binning = pd.merge(schools, student_marks_summary_df, on="school_name", how="left")
del school_summary_binning["School ID"]
school_summary_binning = school_summary_binning.rename(columns={"school_name": "School Name", 
                                                "type": "School Type",
                                                "size": "Total Students",
                                                "budget": "Total School Budget"                                    
                                                })
#school_summary_binning

In [38]:
# check range of values in column
print(school_summary["Per Student Budget"].min())
print(school_summary["Per Student Budget"].max())

$578.00
$655.00


In [39]:
# create bins and labels as bins for scores based on spending ranges
bins = [0, 580, 595, 610, 625, 640, 655]
group_labels = ["$0-$579", "$580-$594", "$595-$609", "$610-$624", "$625-$639", "$640-$656"]

In [40]:
# cut Per Student Budget and place into bins
pd.cut(school_summary_binning["Per Student Budget"], bins, labels=group_labels).head()

0    $625-$639
1    $580-$594
2    $625-$639
3    $640-$656
4    $610-$624
Name: Per Student Budget, dtype: category
Categories (6, object): [$0-$579 < $580-$594 < $595-$609 < $610-$624 < $625-$639 < $640-$656]

In [41]:
# Place the data series into a new column inside of the DataFrame
school_summary_binning["Spending Ranges"] = pd.cut(school_summary_binning["Per Student Budget"], bins, labels=group_labels)
#school_summary_binning

In [42]:
# Create a GroupBy object based upon "Spending Ranges"
school_summary_bin_group = school_summary_binning.groupby("Spending Ranges")

In [43]:
# Find how many rows fall into each bin
print(school_summary_bin_group["Average Math Score"].count())

Spending Ranges
$0-$579      1
$580-$594    3
$595-$609    2
$610-$624    1
$625-$639    4
$640-$656    4
Name: Average Math Score, dtype: int64


In [44]:
# Get the average of each column within the GroupBy object
spending_ranges = school_summary_bin_group[["Average Math Score", 
                      "Average Reading Score", 
                      "% Passing Math", 
                      "% Passing Reading", 
                      "% Overall Passing"]].mean()

## Scores by School Spending

In [45]:
#spending_ranges

### Binning to get scores by school size

In [46]:
# check range of values in column
print(school_summary_binning["Total Students"].min())
print(school_summary_binning["Total Students"].max())

427
4976


In [47]:
# create bins and labels as bins for scores based on school size
bins = [0, 1000, 3000, 5000]
group_labels = ["Small (0-1000)", "Medium (1000-2500)", "Large (2500-5000)"]

In [48]:
# cut Total Students and place into bins
pd.cut(school_summary_binning["Total Students"], bins, labels=group_labels).head()

0     Large (2500-5000)
1    Medium (1000-2500)
2    Medium (1000-2500)
3    Medium (1000-2500)
4    Medium (1000-2500)
Name: Total Students, dtype: category
Categories (3, object): [Small (0-1000) < Medium (1000-2500) < Large (2500-5000)]

In [49]:
# Place the data series into a new column inside of the DataFrame
school_summary_binning["School Size Ranges"] = pd.cut(school_summary_binning["Total Students"], bins, labels=group_labels)
#school_summary_binning

In [50]:
# Create a GroupBy object based upon "School Size Ranges"
school_summary_bin_group = school_summary_binning.groupby("School Size Ranges")

In [51]:
# Find how many rows fall into each bin
print(school_summary_bin_group["Average Math Score"].count())

School Size Ranges
Small (0-1000)        2
Medium (1000-2500)    9
Large (2500-5000)     4
Name: Average Math Score, dtype: int64


In [52]:
# Get the average of each column within the GroupBy object
school_size = school_summary_bin_group[["Average Math Score", 
                      "Average Reading Score", 
                      "% Passing Math", 
                      "% Passing Reading", 
                      "% Overall Passing"]].mean()

## Scores by School Size 

In [53]:
#school_size

### Groupby to get scores by school types

In [54]:
# check for the different object values in column
print(school_summary_binning["School Type"].unique())

['District' 'Charter']


In [55]:
scores_by_school_type = pd.DataFrame({"Average Math Score": school_summary_binning.groupby("School Type") ["Average Math Score"].mean(), 
                                      "Average Reading Score": school_summary_binning.groupby("School Type") ["Average Reading Score"].mean(),
                                      "% Passing Math": pd.to_numeric(school_summary_binning["% Passing Math"]).groupby(school_summary_binning["School Type"]).mean(),
                                      "% Passing Reading": pd.to_numeric(school_summary_binning["% Passing Reading"]).groupby(school_summary_binning["School Type"]).mean(),
                                      "% Overall Passing": pd.to_numeric(school_summary_binning["% Overall Passing"]).groupby(school_summary_binning["School Type"]).mean()
                                   })

## Scores by School Types

In [56]:
#scores_by_school_type